In [35]:
from collections import deque
def bfs(si, sj, ei, ej):
    queue = deque()
    visited = [[0] * N for _ in range(N)]
    queue.append((si, sj))
    visited[si][sj] =((si, sj))
    while queue:
        ci, cj = queue.popleft()
        if (ci, cj) == (ei, ej):
            route = []
            ci, cj = visited[ci][cj]
            while (ci, cj) != (si, sj):
                route.append((ci, cj))
                ci, cj = visited[ci][cj]
            return route[::-1]
        for di, dj in ((-1,0), (1,0), (0,-1), (0,1)):
            ni, nj = ci+di, cj+dj
            if 0<=ni<N and 0<=nj<N and arr[ni][nj] == 0 and visited[ni][nj] == 0:
                visited[ni][nj] = ((ci, cj))
                queue.append((ni, nj))
    return -1

def myprint(arr):
    for lst in arr:
        print(*lst)
    print()
    
def fill_left(v, ni, nj, dr):
    while 0<=ni<N and 0<=nj<N:
        v[ni][nj] = 2
        ni, nj = ni + di[dr], nj + dj[dr]
        
def fill_left_di(v, ni, nj, dr, ori_dr):
    ci, cj = ni + di[dr], nj + dj[dr]
    fill_left(v, ci, cj, dr)
    
    ci, cj = ni + di[ori_dr], nj + dj[ori_dr]
    while 0<=ci<N and 0<=cj<N:
        fill_left(v, ci, cj, dr)
        ci, cj = ci + di[ori_dr], cj + dj[ori_dr]
        
def make_stone(mi, mj, dr, marr):
    v = [[0] * N for _ in range(N)]
    cnt = 0
    # dr 방향으로 > 0 을 만날 때까지 1 표시, 이후 2 표시
    ni, nj = mi + di[dr], mj + dj[dr]
    while 0<=ni<N and 0<=nj<N:
        v[ni][nj] = 1
        if marr[ni][nj] > 0:
            cnt += marr[ni][nj]
            ni, nj = ni + di[dr], nj + dj[dr]
            fill_left(v, ni, nj, dr)
            break
        ni, nj = ni + di[dr], nj + dj[dr]
        
    # dr-1, dr+1 방향으로 원점을 잡고 dr 방향 처리 진행
    for ori_dr in ((dr-1)%8, (dr+1)%8):
        si, sj = mi + di[ori_dr], mj+dj[ori_dr]
        while 0<=si<N and 0<=sj<N:
            if v[si][sj] == 0 and marr[si][sj] > 0:
                v[si][sj] = 1
                cnt += marr[si][sj]
                fill_left_di(v, si, sj, dr, ori_dr)
                break
            
            ci, cj = si, sj
            while 0<=ci<N and 0<=cj<N:
                if v[ci][cj] == 0: # 이전 대각선 탐색 단계에서 방문했을수도 았음
                    v[ci][cj] = 1 
                    if marr[ci][cj] > 0:
                        cnt += marr[ci][cj]
                        fill_left_di(v, ci, cj, dr, ori_dr)
                        break
                else:
                    break
                ci, cj = ci + di[dr], cj + dj[dr]
            si, sj = si + di[ori_dr], sj + dj[ori_dr]
    return v, cnt
            
def move_men(v, mi, mj):
    move_, attk_ = 0, 0
    for dirs in ((0, 4, 6, 2), (6, 2, 0, 4)):        
        for idx in range(len(men)-1, -1, -1):
            ci, cj = men[idx]
            if v[ci][cj] == 1:
                continue
            
            dist = abs(ci-mi) + abs(cj-mj)
            for dr in dirs:
                ni, nj = ci+di[dr], cj+dj[dr]
                if 0<=ni<N and 0<=nj<N and v[ni][nj] != 1 and dist>abs(ni-mi)+abs(nj-mj):                    
                    if (ni, nj) == (mi, mj):
                        attk_ += 1
                        men.pop(idx)
                    else:
                        men[idx] = [ni, nj]
                    move_ += 1
                    break
    return move_, attk_   


In [37]:
# N, M = 4, 4
# si, sj, ei, ej = 1, 3, 3, 3
# men = [[3, 1], [0, 3], [1, 0], [2, 2]]
# arr = [[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1, 1], [1, 0, 0, 0]]

N, M = 6, 4
si, sj, ei, ej = 3, 1, 1, 2
men =[[3,4],[1,4],[0,4],[1,3]]
arr=[[0,0,0,1,0],[0,1,0,0,1,1],[1,1,0,0,0,0],[0,0,1,0,1,1],[0,0,0,0,0,0],[0,0,0,0,1,1]]

route = bfs(si, sj, ei, ej)
if route == -1:
    print(-1)
else:
    for mi, mj in route:
        move_cnt, attk_cnt = 0, 0
        
        # 메두사 이동
        for i in range(len(men)-1, -1, -1):
            if men[i] == [mi,mj]:
                men.pop(i)
            
        # 메두사 시선
        marr = [[0] * N for _ in range(N)]
        for ti, tj in men:
            marr[ti][tj] += 1
        print(mi, mj)
        myprint(marr)
        di = [-1,-1,0,1,1,1,0,-1]
        dj = [ 0, 1,1,1,0,-1,-1,-1]
        mx_stone = -1
        v_all = []
        for dr in (0, 4, 6, 2):
            tv, tstone = make_stone(mi, mj, dr, marr)
            if mx_stone < tstone:
                mx_stone = tstone
                v_all = tv
        myprint(v_all)
        
        # 전사 이동
        # 전사 공격
        move_cnt, attk_cnt = move_men(v_all, mi, mj)
        print(move_cnt, mx_stone, attk_cnt)
    print(0)


4 1
0 0 0 0 1 0
0 0 0 1 1 0
0 0 0 0 0 0
0 0 0 0 1 0
0 0 0 0 0 0
0 0 0 0 0 0

1 1 1 2 2 2
1 1 1 1 1 0
1 1 1 1 0 0
1 1 1 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0

3 2 0
4 2
0 0 0 1 0 0
0 0 0 1 1 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 1 0 0
0 0 0 0 0 0

1 1 1 2 2 2
1 1 1 1 1 1
1 1 1 1 1 0
0 1 1 1 0 0
0 0 0 0 0 0
0 0 0 0 0 0

1 2 1
4 3
0 0 0 1 0 0
0 0 0 1 1 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0

1 1 1 2 2 2
1 1 1 1 1 1
0 1 1 1 1 1
0 0 1 1 1 0
0 0 0 0 0 0
0 0 0 0 0 0

0 2 0
3 3
0 0 0 1 0 0
0 0 0 1 1 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0

1 1 1 2 2 2
0 1 1 1 1 1
0 0 1 1 1 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0

0 2 0
2 3
0 0 0 1 0 0
0 0 0 1 1 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0

0 1 1 2 2 2
0 0 1 1 1 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0

0 2 0
1 3
0 0 0 1 0 0
0 0 0 0 1 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0

0 0 1 1 1 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0
0 0 0 0 0 0

1 1 1
0
